In [11]:
import pandas
titanic = pandas.read_csv("data/train.csv")
print (titanic.describe())

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  


In [12]:
titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())
print (titanic.describe())

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  891.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.361582    0.523008   
std     257.353842    0.486592    0.836071   13.019697    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   22.000000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   35.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  


In [13]:
print (titanic["Sex"].unique())

# Replace all the orrurences of male with the number 0
titanic.loc[titanic["Sex"] == "male", "Sex"] = 0
titanic.loc[titanic["Sex"] == "female", "Sex"] = 1


['male' 'female']


In [14]:
print (titanic["Embarked"].unique())
titanic["Embarked"] = titanic["Embarked"].fillna('S')
titanic.loc[titanic["Embarked"] == "S", "Embarked"] = 0
titanic.loc[titanic["Embarked"] == "C", "Embarked"] = 1
titanic.loc[titanic["Embarked"] == "Q", "Embarked"] = 2
# print(titanic.head())

['S' 'C' 'Q' nan]


In [15]:
# import the linear regression class
from sklearn.linear_model import LinearRegression
# Sklearn also has a helper that makes it easy to do cross validation
from sklearn.cross_validation import KFold

# The columns use to predict the target
predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]

# Initialize our algorithm class
alg = LinearRegression()
# Generate cross validation folds for the titanic dataset
# set_random_state to ensure we get the same splits every time we run it
kf = KFold(titanic.shape[0], n_folds=3, random_state = 1)
# kf = KFold(n_splits=2)
# help(KFold)

predictions= []

for train, test in kf:
        # The predictors we are using the train the algorithm
        train_predictors = (titanic[predictors].iloc[train, :])
        # The target we are using to train the algorithm
        train_target = titanic["Survived"].iloc[train]
        # Training the algorithm using the predictors and target
        alg.fit(train_predictors, train_target)
        # We can now make predictions on the test fold
        test_predictions = alg.predict(titanic[predictors].iloc[test, :])
        predictions.append(test_predictions)


In [16]:
import numpy as np

# The predictions are in three separate numpy arrays. Concatenate them into one
predictions = np.concatenate(predictions, axis = 0)

# Map predictions to outcomes (only possible outcomes are 1 and 0)
predictions[predictions > .5] = 1
predictions[predictions <= .5] = 0
accuracy = sum(predictions[predictions == titanic["Survived"]]) / len(predictions)
print (accuracy)

0.261503928171


In [29]:
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
# Initialize our algorithm
alg = LogisticRegression(random_state = 1)
# Compute the accuracy score for all the cross validation folds
scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic["Survived"], cv = 3)
print(scores.mean())

0.787878787879


In [17]:
# test data clean up
titanic_test = pandas.read_csv("data/test.csv")
titanic_test["Age"] = titanic["Age"].fillna(titanic["Age"].median())
titanic_test["Fare"] = titanic["Fare"].fillna(titanic["Fare"].median())
titanic_test.loc[titanic["Sex"] == "male", "Sex"] = 0
titanic_test.loc[titanic["Sex"] == "female", "Sex"] = 1

titanic_test["Embarked"] = titanic_test["Embarked"].fillna("S")
titanic_test.loc[titanic["Embarked"] == "S", "Embarked"] = 0
titanic_test.loc[titanic["Embarked"] == "C", "Embarked"] = 1
titanic_test.loc[titanic["Embarked"] == "Q", "Embarked"] = 2




In [18]:
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier

predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]

# Initialize our algorithm with the default parameters
# the number of trees we want to make                     --      n_estimators
# the minimum number of rows to split                     --      min_samples_split
# the minimum number of samples when a tree branch ends   --      min_samples_leaf
alg = RandomForestClassifier(random_state = 1, n_estimators = 10, min_samples_split = 2, min_samples_leaf = 1)
# Compute the accuracy score for all the cross validation folds
kf = cross_validation.KFold(titanic.shape[0], n_folds = 3, random_state = 1)
scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic["Survived"], cv = kf)

print(scores.mean())

0.785634118967


In [20]:
# After adjusting random forest parameters
# Several scnerios to prevent like - set the tree too high to overfit
alg = RandomForestClassifier(random_state = 1, n_estimators = 50, min_samples_split = 4, min_samples_leaf = 2)
# Compute the accuracy score for all the cross validation folds
kf = cross_validation.KFold(titanic.shape[0], n_folds = 3, random_state = 1)
scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic["Survived"], cv = kf)

print(scores.mean())

0.81593714927


In [21]:
# More improvements on adding more features
titanic["FamilySize"] = titanic["SibSp"] + titanic["Parch"]

# The apply method generates a new series
titanic["NameLength"] = titanic["Name"].apply(lambda x: len(x))

In [26]:
import re

# A function to get the title from a name
def get_title(name):
    # Use a regular expression to search for a title.
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it
    if title_search:
        return title_search.group()
    return ""

# Extract all the titles and print how often each one occurs
titles = titanic["Name"].apply(get_title)
print(pandas.value_counts(titles))

# Map title with maps
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Dr": 5, "Rev": 6, "Major": 7, "Col": 8, "Mile": 9, "jonkheer": 10, "Lady": 11, "Mme": 12, "Countess": 13, "Capt": 14, "Ms": 15, "Don": 16, "Sir": 17}
for k, v in title_mapping.items():
    titles[titles == k] = v
print(pandas.value_counts(titles))

 Mr.          517
 Miss.        182
 Mrs.         125
 Master.       40
 Dr.            7
 Rev.           6
 Major.         2
 Col.           2
 Mlle.          2
 Jonkheer.      1
 Lady.          1
 Mme.           1
 Countess.      1
 Capt.          1
 Ms.            1
 Don.           1
 Sir.           1
Name: Name, dtype: int64
 Mr.          517
 Miss.        182
 Mrs.         125
 Master.       40
 Dr.            7
 Rev.           6
 Major.         2
 Col.           2
 Mlle.          2
 Jonkheer.      1
 Lady.          1
 Mme.           1
 Countess.      1
 Capt.          1
 Ms.            1
 Don.           1
 Sir.           1
Name: Name, dtype: int64
